In [14]:
pip install scipy


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
#q1 a)

import numpy as np
from scipy.optimize import minimize

def obj_loss_function(params,X,y,lamb):

    w = params[:-1]  
    w0 = params[-1]  

    logits = X @ w + w0  # Xw + w0
    exponent = -y * logits  
    log_loss = np.sum(np.log(1 + np.exp(exponent))) 

    reg_term = (lamb / 2) * np.sum(w**2)
    
    total_loss = log_loss + reg_term
    return total_loss



def minBinDev(X, y, lamb):
    #want to return w, w0
    n, d = X.shape
    
    initial_params = np.zeros(d + 1)  

    results = minimize(obj_loss_function,initial_params,args = (X,y,lamb), method = 'bfgs')

    w_opt = results.x[:-1]  #  d elements are w
    w0_opt = results.x[-1]  # last element is w0
    
    return w_opt, w0_opt
    

    

#  usage
X = np.random.randn(100, 1) 
y = np.random.choice([-1, 1], size=100) 
lamb = 0.1  # Regularization parameter

# Find the optimized weights and bias
w_opt, w0_opt = minBinDev(X, y, lamb)

print("Optimized weights:", w_opt)
print("Optimized bias:", w0_opt) 

Optimized weights: [-0.04883063]
Optimized bias: 0.2835684287674836
